# Download, Unpack, Predict and Save

This notebook downloads the two datasets (fast&slow), all 4x5 trained models and unpacks these.
Then it loads the data and models into memory and runs inferences, saving the results into files.


In [10]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [11]:
import os
# do we have the correct env loaded?
assert 'donkey' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [41]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/

In [41]:
![ ! -d "data/" ] && \
mkdir data && \
curl -L "https://www.dropbox.com/s/k97v7ib2lmyth2u/slower-autodrive.zip?dl=1" --output data/slower-autodrive.zip && \
curl -L "https://www.dropbox.com/s/mt43psjaeozo95m/faster-autodrive.zip?dl=1" --output data/faster-autodrive.zip && \
unzip data/slower-autodrive.zip -d data && \
unzip data/faster-autodrive.zip -d data


In [20]:
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False

DIRECTION = 'CC'
N_FOLDS = 5
DATA_PATH = 'data'


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py


In [21]:
from dataloader import load_records
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive/catalog_20.catalog


19250

In [22]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/slower-autodrive/catalog_20.catalog


20304

In [23]:
from donkeycar.utils import get_model_by_type

model_type = '3d-modified-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
2022-08-16 21:25:01.324462: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d (Conv3D)             (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 10, 26, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [24]:
from losses import get_ground_truth
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


seq size 3
len ground_truth 19250
seq size 3
len ground_truth 20304


In [41]:
![ ! -d "models/" ] && \
mkdir models && \
curl -L "https://www.dropbox.com/s/tsz57xq7i3z6zk8/fast-multi-frame.zip?dl=1" --output models/fast-multi-frame.zip && \
curl -L "https://www.dropbox.com/s/yvitxvv3j8fb0qm/slow-multi-frame.zip?dl=1" --output models/slow-multi-frame.zip && \
unzip models/fast-multi-frame.zip -d models && \
unzip models/slow-multi-frame.zip -d models


In [41]:
from dataloader import get_fold_indices
from losses import get_predictions
from model_loader import load_model
import numpy as np


train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = 'models/fast-multi-frame'

for fold, (train_index, test_index) in enumerate(get_fold_indices(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

In [12]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.2505801434457883, -0.2527056067175679, -0.2393154258884921)

In [13]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((19250,), (19250,), (20304,))

In [14]:
np.save('open-loop-results/nighttime-fast-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.save('open-loop-results/nighttime-fast-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy', np.array([slow_predictions, slow_ground_truths]).T)


In [41]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = 'models/slow-multi-frame'

for fold, (train_index, test_index) in enumerate(get_fold_indices(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

In [16]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.24456062894937428, -0.2491590826787015, -0.25424844947983694)

In [17]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((20304,), (20304,), (19250,))

In [18]:
np.save('open-loop-results/nighttime-slow-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, slow_ground_truths]).T)
np.save('open-loop-results/nighttime-slow-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy', np.array([fast_predictions, fast_ground_truths]).T)


In [33]:
cfg.SEQUENCE_LENGTH = 0
cfg.ROI_CROP_TOP = 59


In [34]:
model_type = 'linear-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter


Model: "linear"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_in (InputLayer)            [(None, 61, 160, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 29, 78, 24)   1824        ['img_in[0][0]']                 
                                                                                                  
 dropout_10 (Dropout)           (None, 29, 78, 24)   0           ['conv2d_1[0][0]']               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 13, 37, 32)   19232       ['dropout_10[0][0]']        

In [35]:
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)

INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive/catalog_20.catalog


19266

In [36]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/slower-autodrive/catalog_20.catalog


20308

In [37]:
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


len ground_truth 19266
len ground_truth 20308


In [30]:
!curl -L "https://www.dropbox.com/s/40alio8uk3hia71/fast-single-frame.zip?dl=1" --output models/fast-single-frame.zip && \
curl -L "https://www.dropbox.com/s/rp5aly3jkjhh1fa/slow-single-frame.zip?dl=1" --output models/slow-single-frame.zip && \
unzip models/fast-single-frame.zip -d models && \
unzip models/slow-single-frame.zip -d models


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   145    0   145    0     0    318      0 --:--:-- --:--:-- --:--:--   329
100   342  100   342    0     0    464      0 --:--:-- --:--:-- --:--:--   464
100 16.6M  100 16.6M    0     0  6955k      0  0:00:02  0:00:02 --:--:-- 11.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   145    0   145    0     0    300      0 --:--:-- --:--:-- --:--:--   306- --:--:-- --:--:--   313
100   342  100   342    0     0    399      0 --:--:-- --:--:-- --:--:--   399
100 16.5M  100 16.5M    0     0  6145k      0  0:00:02  0:00:02 --:--:-- 12.9M
Archive:  models/fast-single-frame.zip
   creating: models/fast-single-frame/
  inflating: models/__MACOSX/._fast-single-frame  
  inflating: models/fast-single-frame/KerasLinearOnlySte

In [41]:

from sklearn.model_selection import KFold

train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = 'models/fast-single-frame'

kf = KFold(n_splits=N_FOLDS, shuffle=False)

for fold, (train_index, test_index) in enumerate(kf.split(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions[:,0]

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

In [24]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.2543695506249286, -0.2543950245587963, -0.24445792576494396)

In [25]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((19266,), (19266,), (20308,))

In [26]:
np.savetxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.savetxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz', np.array([slow_predictions, slow_ground_truths]).T)


In [41]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = 'models/slow-single-frame'

for fold, (train_index, test_index) in enumerate(kf.split(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions[:,0]

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

In [28]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.24557717313088576, -0.24545020701456136, -0.2573492209899343)

In [29]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((20308,), (20308,), (19266,))

In [30]:
np.savetxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, slow_ground_truths]).T)
np.savetxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz', np.array([fast_predictions, fast_ground_truths]).T)
